In [1]:
import numpy
from fractions import Fraction

def analyzePolyOrItsList(inputed):
    if isinstance(inputed, list) or isinstance(inputed, tuple): 
        coefficients = numpy.array(inputed) # 多项式系数(list -> numpy.array)
    elif isinstance(inputed, numpy.poly1d):
        coefficients = inputed.coefficients # 多项式系数(numpy.poly1d -> numpy.array)
    else: return "请输入 numpy.poly1d 多项式或 python 的 list 或 tuple" # 这输入的什么玩意？
    
    degrees = numpy.arange(len(coefficients) - 1, -1, -1) # 生成多项式次数
    total = numpy.sum(coefficients) # 计算所有系数总和
    expectation_fraction = Fraction(numpy.sum(degrees * coefficients), total)  # 期望值分数形式
    expectation = float(expectation_fraction)
    
    print(f"期望值：{expectation_fraction} ≈ {expectation}")
    print(f"总权重：{total}")
    print("| 次数 | 权重 | 分数概率 | 百分比概率 |") # 打印表头
    print("|------|------|----------|------------|")

    for degree, coeff in zip(degrees, coefficients): # 遍历每一项，计算概率并打印
        if coeff == 0:
            continue  # 忽略系数为0的项
        fraction_prob = Fraction(coeff, total)  # 分数概率
        percent_prob = (coeff / total) * 100  # 百分比概率
        print(f"| {degree:4} | {coeff:4} | {fraction_prob:8} | {percent_prob:9.2f}% |")

# 示例：用多项式模拟计算三次骨粉催熟小麦，求小麦age增量的分布
analyzePolyOrItsList(numpy.poly1d([1, 1, 1, 1, 0, 0]) ** 3)


期望值：21/2 ≈ 10.5
总权重：64
| 次数 | 权重 | 分数概率 | 百分比概率 |
|------|------|----------|------------|
|   15 |    1 |     1/64 |      1.56% |
|   14 |    3 |     3/64 |      4.69% |
|   13 |    6 |     3/32 |      9.38% |
|   12 |   10 |     5/32 |     15.62% |
|   11 |   12 |     3/16 |     18.75% |
|   10 |   12 |     3/16 |     18.75% |
|    9 |   10 |     5/32 |     15.62% |
|    8 |    6 |     3/32 |      9.38% |
|    7 |    3 |     3/64 |      4.69% |
|    6 |    1 |     1/64 |      1.56% |


In [2]:
def genBaseDropList(minVal, maxVal):
    if minVal > maxVal: minVal, maxVal = maxVal, minVal # 保证关系为 min < max
    count = maxVal - max(0, minVal) + 1 # 所有权重为1的项，max(min,0)用于截至到0次项
    baseDropList = [0] * (maxVal + 1) # 新建max次多项式，全部初始化为0
    baseDropList[:count] = [1] * count # maxVal~minVal次项系数均设置为1
    baseDropList[-1] = -minVal + 1  if minVal < 0 else baseDropList[-1] # 把负数的权重全部加入0次项
    return baseDropList

def genCommonDropsList(minVal, maxVal, lootVal):
    baseDropList = genBaseDropList(minVal, maxVal)
    if lootVal <= 0 :
        print(f"基础掉落：{minVal} ~ {maxVal}，无掠夺")
        return baseDropList
    else:
        lootDropList = [2] * (lootVal + 1)
        lootDropList[0] = 1
        lootDropList[-1] = 1
        
        commonDropsPoly = numpy.poly1d(baseDropList) * numpy.poly1d(lootDropList)
        commonDropsList = commonDropsPoly.coeffs.tolist()
        print(f"基础掉落：{minVal} ~ {maxVal}，掠夺{lootVal}")
        return commonDropsList

# 示例：凋灵骷髅死亡煤炭基础掉落-1~1个，掠夺3下的掉落煤炭数量分布
a = -1
b = 1
analyzePolyOrItsList(genCommonDropsList(a, b, 0))
analyzePolyOrItsList(genCommonDropsList(a, b, 3))

基础掉落：-1 ~ 1，无掠夺
期望值：1/3 ≈ 0.3333333333333333
总权重：3
| 次数 | 权重 | 分数概率 | 百分比概率 |
|------|------|----------|------------|
|    1 |    1 |      1/3 |     33.33% |
|    0 |    2 |      2/3 |     66.67% |
基础掉落：-1 ~ 1，掠夺3
期望值：11/6 ≈ 1.8333333333333333
总权重：18
| 次数 | 权重 | 分数概率 | 百分比概率 |
|------|------|----------|------------|
|    4 |    1 |     1/18 |      5.56% |
|    3 |    4 |      2/9 |     22.22% |
|    2 |    6 |      1/3 |     33.33% |
|    1 |    5 |     5/18 |     27.78% |
|    0 |    2 |      1/9 |     11.11% |
